In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-15 16:30:40.996093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744734641.010120 1149232 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744734641.014389 1149232 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744734641.028014 1149232 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, os, peft, pickle, pytest, ipytest, pd, transformers, torch, time, threading, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_builder1, ds_builder2, ds_builder3, ds_gs, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_with_strings, map_data, print_trainable_parameters, metric, initialize_heartbeat, heartbeat, end_heartbeat, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

In [2]:
import torch
from opacus.grad_sample import GradSampleModule

In [3]:
lla_321, lla_321_tokenizer=model_from_pkl("Llama-3.2-1B-Instruct")

In [34]:
ds_gst1_train=load_dataset("LongSafari/open-genome", "stage1", split="train[:500]")
#print(ds_gst1[50])
ds_gst1_test=load_dataset("LongSafari/open-genome", "stage1", split="test[:50]")
print(get_dataset_split_names("LongSafari/open-genome", "stage1"))
ds_gst2_train=load_dataset("LongSafari/open-genome", "stage2", split="train[:500]")
ds_gst2_test=load_dataset("LongSafari/open-genome", "stage2", split="test[:50]")
print(get_dataset_split_names("LongSafari/open-genome", "stage2"))
# using a smaller dataset to make sure this works

['train', 'validation', 'test']
['train', 'validation', 'test']


In [39]:
# let's try to use a custom training loop
def preprocess_data(dataset):
    new_dataset=dataset.remove_columns(["text"]) # pytorch does not accept this input
    new_dataset.set_format("torch") # ensures Tensors are returned
    return new_dataset


In [38]:
l_tokenized_stage1_train=preprocess_data(l_tokenized_stage1_train)
l_tokenized_stage1_test=preprocess_data(l_tokenized_stage1_test)

In [ ]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(l_tokenized_stage1_train, shuffle=True, batch_size=8)
test_dataloader=DataLoader(l_tokenized_stage1_test, shuffle=True, batch_size=8)

In [41]:
optimizer=torch.optim.AdamW(lla_lora_model.base_model.parameters(),
                           amsgrad=False, # the AMSGrad variant of this algorithm won't be used 
                            betas=(0.9, 0.999), # coefficients used for computing running averages of gradient and its square
                            capturable=False, # whether the instance will be captured in a CUDA graph
                            differentiable=False, # whether autogad should occur through the optimzer step in training
                            eps=1e-08, # added to denominator to improve numerical stablitity
                            foreach=None, # whether foreach implementation is used
                            fused=None, #whether the fused implementation is used
                            #initial_lr=2e-05,
                            lr=0.1, #learning rate
                            maximize=False, # whether the object is maximized with respect to params instead og
                            weight_decay=0.0)

In [ ]:
# to fine-tune with LoRA, instantiate a base model (as above)
# create LoraConfig where LoRA-specific parameters are defined
config=LoraConfig(
    #inference_mode=False,
    r=8, #rank of update matrices, lower value results in smaller matrices with fewer parameters
    lora_alpha=16, #LoRA scaling factor
    task_type="CAUSAL_LM",
    lora_dropout=0.1, # dropout probability of LoRA layers
    bias="none", # specifies if bias parameters should be trained
    #modules_to_save=["decode_head"] #models apart from LoRA layers that are trainable
)

In [19]:
lla_lora_model=get_peft_model(lla_321, config)

In [ ]:
l3_trainer=make_trainer(lla_lora_model, train_dataloader.dataset, test_dataloader.dataset, config,
                          SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ), (optimizer))

In [ ]:
from transformers import get_scheduler
def dp_train(trainer):
    num_epochs=trainer.args.num_train_epochs
    num_steps=num_epochs * len(train_dataloader)
    scheduler=get_scheduler(
                name="linear",
                optimizer=self.optimizer,
                num_warmup_steps=0,
                num_training_steps=num_training_steps,
            ) #this is the default learning rate scheduler from the trainer
    

In [ ]:
if torch.cuda.is_available():
    device="cuda:0" 

lla_lora_model=lla_lora_model.to(device)

In [24]:
l3_trainer.train()

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.678200,2.678609


/home/ac.cdavies/.vnv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=125, training_loss=0.678159423828125, metrics={'train_runtime': 34.2864, 'train_samples_per_second': 14.583, 'train_steps_per_second': 3.646, 'total_flos': 2992122101760000.0, 'train_loss': 0.678159423828125})

In [ ]:
l3_trainer.state.log_history[0]['loss']

In [ ]:
l3_trainer.train_dataset

In [ ]:
dir(l3_trainer.prediction_step)

In [ ]:
# the Trainer object will call the optimizer's train() function at each training step if method is callable/exists
# maybe create a train method for your optimizer with the required steps for DPOptimizer 
def optimize(trainer):
    #for i in range(l3_trainer.args.num_train_epochs): might not be needed if the trainer executes this at steps/epochs
    losses=[] # empty loss list
    data_loader=trainer.get_train_dataloader
    optimizer=trainer.optimizer
    optimizer.zero_grad() # resets the gradients of the parameter tensors to zero

    outputs= trainer.#training output at this step/epoch
    loss=outputs[0]['loss'] #appropriate position in outputs
    loss.backward() # calculates gradient by backpropagating error from the current loss
    losses.append(loss.item()) #add the gradient value to the list
    optimizer.step() # perform optimization step to update the parameter
    
    

In [ ]:
optimizer.step

In [ ]:
optimizer.pre_step

In [ ]:
optimizer.train=optimize

In [ ]:
from opacus import PrivacyEngine
privacy_engine = PrivacyEngine(secure_mode=False)
model, optimizer, train_loader = privacy_engine.make_private(
        module=lla_lora_model,
        optimizer=optimizer,
        data_loader=train_dataloader,
        noise_multiplier=1.3,
        max_grad_norm=1.0,
        )

In [ ]:
lla_lora_model=get_peft_model(lla_321, config)

In [ ]:
l6_trainer=make_trainer(lla_lora_model, train_dataloader.dataset, test_dataloader.dataset, config,
                          SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6,
                                    ), optimizer)

In [ ]:
l6_trainer.train()

In [ ]:
dir(optimizer)

In [ ]:
l6_trainer.optimizer.train(l5_trainer)

In [ ]:
l5_trainer.train()

In [ ]:
dir(l3_trainer.optimizer)

In [ ]:
type(l3_trainer)

In [ ]:
# run the .get_optimizer_cls_and_kwargs on the trainer
l3_trainer.get_optimizer_cls_and_kwargs(SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ))

In [ ]:
l4_trainer=make_trainer(lla_lora_model, train_dataloader.dataset, test_dataloader.dataset, config,
                          SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ))

In [ ]:
l4_trainer.train()

In [ ]:
dir(l4_trainer.optimizer)
# it seems that the grad_samples parameter doesn't exist in a regular AdamW optimizer, it is just for the DPOptimizer version
# when the .get_optimizer_cls_and_kwargs function is ran on the trainer with the DPOptimizer, it retrieves arguments from the normal AdamW optimizer

In [ ]:
l4_trainer.get_optimizer_cls_and_kwargs(SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ))
# for the default and imported optimizers, the same arguments are retrieved by this function